📖 Scenario:
At a small college called Coffee & Courses, students enroll in various majors and often enjoy drinks at the campus café. We'll create a database with two tables:

students – Student details

purchases – Drink purchase history at the café



🧱 SQL: Create Tables & Insert Sample Data


In [1]:
import sqlite3
import pandas as pd

In [2]:
# Create a connection to the SQLite3 database
conn = sqlite3.connect('coffee_courses.db')
cursor = conn.cursor()

# Create Students Table
# This table stores information about students.
# Fields:
# - student_id: A unique identifier for each student (Primary Key).
# - name: The name of the student.
# - major: The major subject the student is enrolled in.
# - year: The current year of study for the student.
cursor.execute('''
CREATE TABLE IF NOT EXISTS students (
  student_id INTEGER PRIMARY KEY,
  name TEXT,
  major TEXT,
  year INTEGER
);
''')

# Create Purchases Table
# This table records the purchase history of drinks at the café.
# Fields:
# - purchase_id: A unique identifier for each purchase (Primary Key).
# - student_id: The ID of the student who made the purchase (Foreign Key referencing students table).
# - item: The name of the drink purchased.
# - price: The cost of the drink.
# - purchase_date: The date when the purchase was made.
cursor.execute('''
CREATE TABLE IF NOT EXISTS purchases (
  purchase_id INTEGER PRIMARY KEY,
  student_id INTEGER,
  item TEXT,
  price REAL,
  purchase_date TEXT,
  FOREIGN KEY(student_id) REFERENCES students(student_id)
);
''')

# Insert Sample Students
# Adding sample data to the students table.
cursor.executemany('''
INSERT INTO students (student_id, name, major, year) VALUES (?, ?, ?, ?)
''', [
    (1, 'Alice', 'Computer Science', 2),
    (2, 'Bob', 'Economics', 3),
    (3, 'Charlie', 'Psychology', 1),
    (4, 'Dana', 'Computer Science', 3),
    (5, 'Eli', 'Economics', 2)
])

# Insert Sample Purchases
# Adding sample data to the purchases table.
cursor.executemany('''
INSERT INTO purchases (purchase_id, student_id, item, price, purchase_date) VALUES (?, ?, ?, ?, ?)
''', [
    (101, 1, 'Espresso', 2.5, '2025-05-20'),
    (102, 1, 'Latte', 3.5, '2025-05-21'),
    (103, 2, 'Cappuccino', 4.0, '2025-05-20'),
    (104, 3, 'Tea', 2.0, '2025-05-22'),
    (105, 4, 'Latte', 3.5, '2025-05-22'),
    (106, 5, 'Espresso', 2.5, '2025-05-22'),
    (107, 5, 'Latte', 3.5, '2025-05-23'),
    (108, 4, 'Tea', 2.0, '2025-05-24')
])

# Commit the changes and close the connection
conn.commit()


🎓 SQL Exercises by Topic


### 1.List all students in their 2nd year


In [3]:
query = """
SELECT *
FROM students
WHERE year = 2
"""
df = pd.read_sql_query(query, conn)
print(df)


   student_id   name             major  year
0           1  Alice  Computer Science     2
1           5    Eli         Economics     2


### 2.Which students are majoring in Computer Science?


In [5]:
query = """
SELECT *
FROM students
WHERE major = 'Computer Science'
"""
df = pd.read_sql_query(query, conn)
print(df)

   student_id   name             major  year
0           1  Alice  Computer Science     2
1           4   Dana  Computer Science     3


### 3.What purchases were made on '2025-05-22'?


In [9]:
query = """
SELECT item
FROM purchases
WHERE purchase_date = '2025-05-22'
"""
df = pd.read_sql_query(query, conn)
print(df)

       item
0       Tea
1     Latte
2  Espresso


### 4. Which items cost more than 3?


In [10]:
query = """
SELECT item
FROM purchases
WHERE price > 3
"""
df = pd.read_sql_query(query, conn)
print(df)

         item
0       Latte
1  Cappuccino
2       Latte
3       Latte


### 5. Show each student's name and what they bought



In [11]:
query = """
SELECT students.name, purchases.item
FROM students
JOIN purchases ON students.student_id = purchases.student_id
"""
df = pd.read_sql_query(query, conn)
print(df)

      name        item
0    Alice    Espresso
1    Alice       Latte
2      Bob  Cappuccino
3  Charlie         Tea
4     Dana       Latte
5      Eli    Espresso
6      Eli       Latte
7     Dana         Tea


### 6. Who purchased a "Latte"?


In [17]:
query = """
SELECT students.name, purchases.item
FROM students
JOIN purchases ON students.student_id = purchases.student_id 
WHERE item = 'Latte'
"""
df = pd.read_sql_query(query, conn)
print(df)

    name   item
0  Alice  Latte
1   Dana  Latte
2    Eli  Latte


### 7. Show each student’s name, major, and price of each purchase:




In [18]:
query = """
SELECT students.name, students.major, purchases.price
FROM students
JOIN purchases ON students.student_id = purchases.student_id
"""
df = pd.read_sql_query(query, conn)
print(df)

      name             major  price
0    Alice  Computer Science    2.5
1    Alice  Computer Science    3.5
2      Bob         Economics    4.0
3  Charlie        Psychology    2.0
4     Dana  Computer Science    3.5
5      Eli         Economics    2.5
6      Eli         Economics    3.5
7     Dana  Computer Science    2.0


### 8. How much has each student spent in total?






In [22]:
query = """
SELECT students.name, SUM(purchases.price) 
FROM students
JOIN purchases ON students.student_id = purchases.student_id
GROUP BY students.name
"""
df = pd.read_sql_query(query, conn)
print(df)

      name  SUM(purchases.price)
0    Alice                   6.0
1      Bob                   4.0
2  Charlie                   2.0
3     Dana                   5.5
4      Eli                   6.0


### 9. How many purchases were made per drink type?







In [26]:
query = """
SELECT purchases.item, COUNT(*)
FROM purchases
GROUP BY purchases.item
"""
df = pd.read_sql_query(query, conn)
print(df)

         item  COUNT(*)
0  Cappuccino         1
1    Espresso         2
2       Latte         3
3         Tea         2


### 10. What is the average price of each drink?






In [32]:
query = """
SELECT item, AVG(price) as avg_price
FROM purchases
GROUP BY item
"""
df = pd.read_sql_query(query, conn)
print(df)

         item  avg_price
0  Cappuccino        4.0
1    Espresso        2.5
2       Latte        3.5
3         Tea        2.0


### 11. How many students are there in each major?






In [33]:
query = """
SELECT students.major, COUNT(*)
FROM students
GROUP BY major
"""
df = pd.read_sql_query(query, conn)
print(df)

              major  COUNT(*)
0  Computer Science         2
1         Economics         2
2        Psychology         1


### 12. Which students spent more than 3 total?








In [35]:
query = """
SELECT students.name, SUM(purchases.price) as total_spent
FROM students
JOIN purchases ON students.student_id = purchases.student_id
GROUP BY students.name
HAVING total_spent > 3
"""
df = pd.read_sql_query(query, conn)
print(df)

    name  total_spent
0  Alice          6.0
1    Bob          4.0
2   Dana          5.5
3    Eli          6.0


### 13. Which drinks were purchased at least 3 times?






In [37]:
query = """
SELECT item, COUNT(*) as purchase_count
FROM purchases
GROUP BY item
HAVING COUNT(*) >= 3
"""
df = pd.read_sql_query(query, conn)
print(df)

    item  purchase_count
0  Latte               3


### 14. Show all purchases ordered by price (descending):








In [41]:
query = """
SELECT purchases.item, purchases.price
FROM purchases
ORDER BY price DESC
"""
df = pd.read_sql_query(query, conn)
print(df)

         item  price
0  Cappuccino    4.0
1       Latte    3.5
2       Latte    3.5
3       Latte    3.5
4    Espresso    2.5
5    Espresso    2.5
6         Tea    2.0
7         Tea    2.0


### 15. List student names sorted by academic year:







In [42]:
query = """
SELECT students.name, students.year
FROM students
ORDER BY year
"""
df = pd.read_sql_query(query, conn)
print(df)

      name  year
0  Charlie     1
1    Alice     2
2      Eli     2
3      Bob     3
4     Dana     3


### 16. Show top 3 spenders:







In [47]:
query = """
SELECT students.name, SUM(purchases.price) as "total spend"
FROM students
JOIN purchases ON students.student_id = purchases.student_id
GROUP BY students.name
ORDER BY "total spend" DESC
LIMIT 3
"""
df = pd.read_sql_query(query, conn)
print(df)

    name  total spend
0    Eli          6.0
1  Alice          6.0
2   Dana          5.5


In [48]:
conn.close()